In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedBobDecoder(nn.Module):
    """Bob decoder with key-dependent decryption (mirrors Alice)"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128):
        super(ImprovedBobDecoder, self).__init__()

        self.vocab_size = vocab_size

        # Key decryption (reverse of Alice's encryption)
        from v3_part2_alice import MultiLayerKeyEncryption
        self.key_decryption = MultiLayerKeyEncryption(embed_dim, num_key_layers=4)

        # Decoder layers
        self.decoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=4,
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            ) for _ in range(num_layers)
        ])

        # Output projection
        self.output_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embed_dim * 2, vocab_size)
        )

    def forward(self, encrypted, keys):
        """
        encrypted: (B, L, D)
        keys: list of (B, L) tensors
        """
        # Apply key-dependent decryption
        x = self.key_decryption(encrypted, keys)

        # Transformer decoding
        for layer in self.decoder_layers:
            x = layer(x)

        # Predict tokens
        logits = self.output_proj(x)

        return logits


class StrongerEveAttacker(nn.Module):
    """Eve with NO access to keys - must learn patterns directly"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=6, max_len=128):
        super(StrongerEveAttacker, self).__init__()

        # Eve uses MORE layers but NO key information
        self.input_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(embed_dim * 2, embed_dim),
            nn.LayerNorm(embed_dim)
        )

        # Deep transformer (no keys!)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=8,
                dim_feedforward=embed_dim * 4,
                dropout=0.2,
                activation='gelu',
                batch_first=True
            ) for _ in range(num_layers)
        ])

        # Output head
        self.output_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(embed_dim * 2, vocab_size)
        )

        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if module.bias is not None:
                    module.bias.data.zero_()

    def forward(self, encrypted):
        """
        encrypted: (B, L, D)
        NO keys - Eve is blind to key information!
        """
        x = self.input_proj(encrypted)

        # Deep processing without keys
        for layer in self.transformer_layers:
            x = layer(x)

        # Predict
        logits = self.output_head(x)

        return logits


class BobDecryptor:
    """Wrapper for Bob decoder"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.bob = ImprovedBobDecoder(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v3_part1 import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def decrypt(self, encrypted, keys):
        """Decrypt with keys"""
        seq_len = encrypted.size(1)
        truncated_keys = [k[:, :seq_len] for k in keys['key_tensors']]

        self.bob.eval()
        with torch.no_grad():
            logits = self.bob(encrypted, truncated_keys)
            tokens = torch.argmax(logits, dim=-1)

        messages = self.processor.batch_decode(tokens)

        return messages, tokens, logits


class EveAttacker:
    """Wrapper for Eve attacker"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=6, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.eve = StrongerEveAttacker(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v3_part1 import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def attack(self, encrypted):
        """Attack without keys"""
        self.eve.eval()
        with torch.no_grad():
            logits = self.eve(encrypted)
            tokens = torch.argmax(logits, dim=-1)

        messages = self.processor.batch_decode(tokens)

        return messages, tokens, logits

In [4]:

if __name__ == "__main__":
    print("="*70)
    print("IMPROVED BOB DECODER & EVE ATTACKER V3")
    print("="*70)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    from v3_part1 import StringProcessor, KeyManager

    processor = StringProcessor(max_length=64)
    bob_dec = BobDecryptor(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=4,
        max_len=64,
        device=device
    )
    eve_att = EveAttacker(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=6,
        max_len=64,
        device=device
    )

    # Simulate encrypted data
    batch_size = 3
    seq_len = 64
    embed_dim = 128
    encrypted = torch.randn(batch_size, seq_len, embed_dim).to(device)

    key_manager = KeyManager(device=device)
    keys = key_manager.generate_keys_for_batch(batch_size, seq_len)

    print("\nBob decrypting with correct keys...")
    bob_messages, _, _ = bob_dec.decrypt(encrypted, keys)
    print("Bob's outputs:")
    for i, msg in enumerate(bob_messages):
        print(f"  {i+1}. '{msg}'")

    print("\nEve attacking without keys...")
    eve_messages, _, _ = eve_att.attack(encrypted)
    print("Eve's outputs:")
    for i, msg in enumerate(eve_messages):
        print(f"  {i+1}. '{msg}'")

    print("\n" + "="*70)

IMPROVED BOB DECODER & EVE ATTACKER V3

Bob decrypting with correct keys...
Bob's outputs:
  1. '66?A]}WA(.(6?*I6W'
  2. 'X(W8('
  3. ''

Eve attacking without keys...
Eve's outputs:
  1. 'tc)Y9\kcTdYT4ttct9PTPTdt9h65ePku@ut`Vtu`\ceP3c[>VtcT?Ycucd/tu'
  2. 'c?,PYchcPtPP{\tt1thQ!t[pc/hccttt["cPPtTtkdPTPtct4cgcLPRYtI[lcPc'
  3. '[#dH#\T[\->\ucYuPuP/#H-PePHutIY@uct%9Pgt\dtYY\ttuVZdP`ectutY`c'

